<a href="https://colab.research.google.com/github/jh9553-commits/CUSTOMER-CHURN-PREDICTION-WITH-PROFIT-MAXIMIZATION-/blob/main/Data_bootcamp_final_project_without_Visualizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
CUSTOMER CHURN PREDICTION WITH PROFIT MAXIMIZATION
Comprehensive Analysis with Hyperparameter Tuning and Statistical Validation

Author: Jianxun Huang
Final Version - December 19, 2025

EXECUTIVE SUMMARY:
This analysis predicts customer churn for a telecom company and optimizes retention strategies
using cost-sensitive machine learning. By tuning two models (HistGradientBoosting and Neural Network)
and applying threshold optimization + customer segmentation, we achieve $146,893 in annual profit—
a 186% improvement over baseline approaches that use standard classification thresholds.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score,
    precision_score, recall_score, f1_score, accuracy_score,
    roc_curve, auc, precision_recall_curve
)
from scipy import stats
import warnings
warnings.filterwarnings('ignore')


# ============================================================================
# UTILITY FUNCTIONS - DATA PROCESSING
# ============================================================================

class DataProcessor:
    """
    Handles data loading and preprocessing operations.

    Purpose: Centralize all data preparation logic to ensure consistency
    and make the pipeline reproducible and maintainable.
    """

    @staticmethod
    def load_and_preprocess(filepath):
        """
        Load customer churn dataset and prepare features for modeling.

        Process:
        1. Load CSV file from disk
        2. Remove non-informative identifier column (customerID)
        3. Convert TotalCharges to numeric (handles edge case of string values)
        4. Impute missing charges with median (statistically sound approach)
        5. Separate features (X) and target (y)
        6. Encode categorical variables to numeric format (required by ML models)

        Returns:
            X (DataFrame): 7,043 rows × 19 feature columns
            y (Series): Binary churn labels (0=No, 1=Yes)
            df (DataFrame): Original data for reference
        """
        # Load raw CSV
        df = pd.read_csv(filepath)

        # Remove customerID (not useful for prediction, only identifies rows)
        df = df.drop('customerID', axis=1)

        # Clean TotalCharges: convert to numeric and handle missing values
        # Some rows may have whitespace/non-numeric values in TotalCharges
        df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
        # Impute missing values with median (robust to outliers)
        df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

        # Separate target and features
        X = df.drop('Churn', axis=1)
        # Convert Churn from 'Yes'/'No' strings to 1/0 binary
        y = (df['Churn'] == 'Yes').astype(int)

        # Encode all categorical features (one-hot encoding or label encoding)
        # LabelEncoder maps each category to unique integer (0, 1, 2, ...)
        for col in X.select_dtypes(include='object').columns:
            X[col] = LabelEncoder().fit_transform(X[col])

        return X, y, df


# ============================================================================
# UTILITY FUNCTIONS - METRICS CALCULATION
# ============================================================================

class MetricsCalculator:
    """
    Computes comprehensive classification metrics for model evaluation.

    Purpose: Provide standardized metrics calculation for consistent model
    comparison across different thresholds and strategies.
    """

    @staticmethod
    def calculate_metrics(y_true, y_proba, y_pred, model_name, threshold=0.5):
        """
        Calculate all classification metrics from predicted probabilities and binary predictions.

        Key Metrics:
        - Accuracy: (TP + TN) / Total — overall correctness (can be misleading for imbalanced data)
        - Precision: TP / (TP + FP) — when we predict churn, how often are we right?
        - Recall: TP / (TP + FN) — of actual churners, how many do we catch?
        - Specificity: TN / (TN + FP) — of actual non-churners, how many do we correctly identify?
        - F1-Score: 2 * (Precision * Recall) / (Precision + Recall) — harmonic mean balancing both
        - ROC-AUC: Area under ROC curve [0-1] — probability the model ranks random churn > non-churn
        - MCC: Matthews Correlation Coefficient [-1, 1] — correlation coefficient for binary classification
        - Balanced Accuracy: (Recall + Specificity) / 2 — average of recall and specificity

        Args:
            y_true: Actual binary labels (0/1)
            y_proba: Predicted probability of churn [0-1] from model
            y_pred: Binary predictions (0/1) based on threshold
            model_name: Name of model (e.g., 'HistGradientBoost')
            threshold: Decision threshold used for y_pred

        Returns:
            Dictionary with all computed metrics for easy dataframe conversion
        """
        # Extract confusion matrix components from sklearn
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

        # Calculate individual metrics from confusion matrix
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # ROC-AUC: threshold-independent metric that works well with imbalanced data
        roc_auc = roc_auc_score(y_true, y_proba)

        # Matthews Correlation Coefficient: single score accounting for all confusion matrix elements
        mcc_denom = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
        mcc = ((tp * tn) - (fp * fn)) / mcc_denom if mcc_denom > 0 else 0

        # Balanced Accuracy: useful for imbalanced datasets
        balanced_acc = (recall + specificity) / 2

        return {
            'Model': model_name,
            'Threshold': threshold,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'Specificity': specificity,
            'F1_Score': f1,
            'ROC_AUC': roc_auc,
            'MCC': mcc,
            'Balanced_Accuracy': balanced_acc,
            'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn
        }

    @staticmethod
    def generate_profit_function(y_true, y_proba, cost_matrix):
        """
        Generate profit values across decision threshold range [0.05, 0.95].

        Business Logic:
        Different decision thresholds lead to different TP/FP/FN/TN distributions,
        which directly impact profitability given the cost matrix:
        - TP: We retain a churner (revenue saved) → positive value
        - FP: We waste money on unnecessary retention → negative value
        - FN: We miss a churner (lost customer value) → most negative value
        - TN: Correctly identified non-churner (no action) → zero cost

        This function finds the threshold that maximizes total profit across all customers.

        Args:
            y_true: Actual binary labels
            y_proba: Predicted probability of churn
            cost_matrix: Dict with keys 'TP', 'FP', 'FN', 'TN' containing financial values

        Returns:
            results: List of dicts with profit/precision/recall at each threshold
            optimal_threshold: Threshold that maximizes profit
        """
        # Evaluate 100 evenly-spaced thresholds from 0.05 to 0.95
        thresholds = np.linspace(0.05, 0.95, 100)
        results = []

        for threshold in thresholds:
            # Convert probabilities to binary predictions using this threshold
            # Example: if threshold=0.3, predict churn if probability > 0.3
            y_pred = (y_proba > threshold).astype(int)

            # Compute confusion matrix for this threshold
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

            # Calculate total profit (sum of all individual customer outcomes)
            profit = (tp * cost_matrix['TP'] +
                     fp * cost_matrix['FP'] +
                     fn * cost_matrix['FN'] +
                     tn * cost_matrix['TN'])

            # Also compute standard metrics at this threshold for analysis
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

            # Store all results for this threshold
            results.append({
                'threshold': threshold,
                'profit': profit,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'tp': tp, 'fp': fp, 'fn': fn, 'tn': tn
            })

        # Find threshold with maximum profit
        max_profit = max([r['profit'] for r in results])
        optimal_idx = [r['profit'] for r in results].index(max_profit)
        optimal_threshold = thresholds[optimal_idx]

        return results, optimal_threshold


# ============================================================================
# UTILITY FUNCTIONS - STATISTICAL ANALYSIS
# ============================================================================

class StatisticalAnalyzer:
    """
    Performs statistical significance testing and confidence intervals.

    Purpose: Validate that observed model differences are real and not due to
    chance, using rigorous statistical methods.
    """

    @staticmethod
    def bootstrap_auc(y_true, y_proba, n_iterations=1000, ci=95):
        """
        Bootstrap resampling for ROC-AUC confidence intervals.

        Rationale: With a single test set, we get a point estimate of AUC.
        Bootstrap estimates the sampling distribution by resampling with replacement
        1000 times, allowing us to quantify uncertainty in the AUC estimate.

        Args:
            y_true: Actual labels
            y_proba: Predicted probabilities
            n_iterations: Number of bootstrap samples (1000 is standard)
            ci: Confidence interval percentage (95% is standard)

        Returns:
            Dictionary with:
            - AUC_Mean: Central estimate
            - AUC_StdDev: Standard deviation
            - CI_Lower: Lower bound of 95% CI
            - CI_Upper: Upper bound of 95% CI
        """
        bootstrap_aucs = []
        n = len(y_true)

        # Resample with replacement 1000 times
        for _ in range(n_iterations):
            # Randomly select indices with replacement (some repeated, some missing)
            indices = np.random.choice(n, n, replace=True)
            y_true_boot = y_true.iloc[indices] if hasattr(y_true, 'iloc') else y_true[indices]
            y_proba_boot = y_proba[indices]

            # Only compute AUC if we have both classes in the bootstrap sample
            if len(np.unique(y_true_boot)) == 2:
                auc_boot = roc_auc_score(y_true_boot, y_proba_boot)
                bootstrap_aucs.append(auc_boot)

        # Compute summary statistics from bootstrap distribution
        auc_mean = np.mean(bootstrap_aucs)
        auc_std = np.std(bootstrap_aucs)
        # Use percentile method for CI (more robust than normal approximation)
        lower = np.percentile(bootstrap_aucs, (100 - ci) / 2)
        upper = np.percentile(bootstrap_aucs, 100 - (100 - ci) / 2)

        return {
            'AUC_Mean': auc_mean,
            'AUC_StdDev': auc_std,
            'CI_Lower': lower,
            'CI_Upper': upper
        }

    @staticmethod
    def paired_ttest_analysis(y_test, y_proba_hist, y_proba_hf):
        """
        Paired t-test comparing F1-scores across multiple thresholds.

        Rationale: Both models produce predictions on the same test set.
        A paired t-test measures if the difference in average F1-score is
        statistically significant, not just due to random variation.

        Interpretation:
        - p-value < 0.05: Significant difference (reject null hypothesis of equality)
        - p-value >= 0.05: No significant difference (cannot reject equality)

        Args:
            y_test: Test set labels
            y_proba_hist: HistGradientBoost probabilities on test set
            y_proba_hf: Neural Network probabilities on test set

        Returns:
            Dictionary with means, std devs, t-statistic, and p-value
        """
        # Compute F1 scores at 50 different thresholds for each model
        thresholds = np.linspace(0.1, 0.9, 50)
        hist_f1_scores = []
        hf_f1_scores = []

        for t in thresholds:
            # Convert probabilities to binary predictions at threshold t
            hist_pred_t = (y_proba_hist > t).astype(int)
            hf_pred_t = (y_proba_hf > t).astype(int)

            # Compute F1 score at this threshold for each model
            hist_f1 = f1_score(y_test, hist_pred_t, zero_division=0)
            hf_f1 = f1_score(y_test, hf_pred_t, zero_division=0)

            hist_f1_scores.append(hist_f1)
            hf_f1_scores.append(hf_f1)

        # Convert to numpy arrays for statistical testing
        hist_f1_scores = np.array(hist_f1_scores)
        hf_f1_scores = np.array(hf_f1_scores)

        # Perform paired t-test (compares paired samples)
        # H0: Mean F1 difference = 0 (models perform equally)
        # Ha: Mean F1 difference ≠ 0 (models differ)
        t_stat, p_value = stats.ttest_rel(hf_f1_scores, hist_f1_scores)

        return {
            'HistGB_Mean_F1': hist_f1_scores.mean(),
            'HistGB_Std_F1': hist_f1_scores.std(),
            'NN_Mean_F1': hf_f1_scores.mean(),
            'NN_Std_F1': hf_f1_scores.std(),
            't_statistic': t_stat,
            'p_value': p_value
        }


# ============================================================================
# MAIN ANALYSIS PIPELINE
# ============================================================================

def print_section_header(title):
    """Print formatted section header for readability."""
    print("\n" + "="*100)
    print(title.center(100))
    print("="*100)


def print_subsection(subtitle):
    """Print formatted subsection header."""
    print("\n" + subtitle)
    print("-"*100)


def main():
    """
    Main analysis pipeline orchestrating all analysis steps.

    Pipeline flow:
    1. Data preparation (load, preprocess, train/test split)
    2. Hyperparameter tuning (GridSearchCV for both models)
    3. Classification metrics (default 0.50 threshold)
    4. Statistical validation (bootstrap, t-test, cross-validation)
    5. Profit optimization (threshold tuning)
    6. Segmentation strategy (segment-based profit maximization)
    7. Report generation (CSV outputs for presentation)
    """

    print_section_header("CUSTOMER CHURN PREDICTION ANALYSIS")
    print_section_header("Data Preparation and Feature Engineering")

    # ========================================================================
    # STEP 1: DATA LOADING AND PREPROCESSING
    # ========================================================================

    # Load and preprocess the customer churn dataset
    X, y, df = DataProcessor.load_and_preprocess('WA_Fn-UseC_-Telco-Customer-Churn.csv')

    # Split data: 80% training, 20% testing with stratification (preserves class ratio)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Print dataset summary
    print_subsection("Dataset Summary")
    print(f"Total observations: {len(df):,}")
    print(f"Total features: {X.shape[1]}")
    print(f"Target variable (Churn): Binary classification")
    print(f"Churn rate: {y.mean()*100:.2f}%")
    print(f"Class imbalance ratio: {(1-y.mean())/y.mean():.2f}:1")
    print(f"Training set size: {len(X_train):,}")
    print(f"Testing set size: {len(X_test):,}")

    # ========================================================================
    # STEP 2: HYPERPARAMETER TUNING - HISTGRADIENTBOOSTING
    # ========================================================================

    print_section_header("Hyperparameter Optimization")

    print_subsection("HistGradientBoosting Classifier Tuning")

    # Define grid of hyperparameters to search (81 combinations = 3×3×3×3)
    hist_param_grid = {
        'max_iter': [150, 200, 250],           # Number of boosting iterations
        'learning_rate': [0.05, 0.1, 0.15],    # Step size for each iteration
        'max_depth': [8, 10, 12],               # Maximum tree depth
        'l2_regularization': [0.5, 1.0, 1.5]   # Penalty for model complexity
    }

    # GridSearchCV: evaluates all parameter combinations using 3-fold cross-validation
    # Scoring on ROC-AUC (threshold-independent, better for imbalanced data)
    hist_grid = GridSearchCV(
        HistGradientBoostingClassifier(min_samples_leaf=10, random_state=42),
        hist_param_grid,
        cv=3,                  # 3-fold cross-validation
        n_jobs=-1,             # Use all available CPU cores
        scoring='roc_auc',     # Optimize ROC-AUC
        verbose=0              # Suppress output
    )

    # Fit grid search on training data (trains 81 * 3 = 243 models)
    hist_grid.fit(X_train, y_train)
    hist_model = hist_grid.best_estimator_

    print(f"Grid search completed: 81 parameter combinations evaluated")
    print(f"Cross-validation strategy: 3-fold stratified")
    print(f"Optimal ROC-AUC score: {hist_grid.best_score_:.4f}")
    print(f"Best parameters: {hist_grid.best_params_}")

    # Generate predictions on test set with best model
    y_proba_hist = hist_model.predict_proba(X_test)[:, 1]  # Probability of churn
    y_pred_hist = (y_proba_hist > 0.5).astype(int)         # Binary predictions at default 0.5

    # ========================================================================
    # STEP 3: HYPERPARAMETER TUNING - NEURAL NETWORK
    # ========================================================================

    print_subsection("Neural Network Classifier Tuning")

    # Define grid of hyperparameters for MLP (81 combinations)
    hf_param_grid = {
        'hidden_layer_sizes': [(64, 32), (128, 64, 32), (128, 64, 32, 16)],  # Network architecture
        'learning_rate_init': [0.0005, 0.001, 0.002],                         # Initial learning rate
        'alpha': [0.00005, 0.0001, 0.0002],                                   # L2 regularization
        'max_iter': [250, 300, 350]                                           # Maximum iterations
    }

    # GridSearchCV for Neural Network with early stopping
    hf_grid = GridSearchCV(
        MLPClassifier(
            early_stopping=True,        # Stop if validation error plateaus
            validation_fraction=0.1,    # Use 10% of training for validation
            random_state=42,
            n_iter_no_change=20         # Patience: stop after 20 iterations without improvement
        ),
        hf_param_grid,
        cv=3,
        n_jobs=-1,
        scoring='roc_auc',
        verbose=0
    )

    # Fit grid search (trains 81 * 3 = 243 models with early stopping)
    hf_grid.fit(X_train, y_train)
    hf_model = hf_grid.best_estimator_

    print(f"Grid search completed: 81 parameter combinations evaluated")
    print(f"Cross-validation strategy: 3-fold stratified with early stopping")
    print(f"Optimal ROC-AUC score: {hf_grid.best_score_:.4f}")
    print(f"Best parameters: {hf_grid.best_params_}")

    # Generate predictions on test set with best model
    y_proba_hf = hf_model.predict_proba(X_test)[:, 1]  # Probability of churn
    y_pred_hf = (y_proba_hf > 0.5).astype(int)         # Binary predictions at default 0.5

    # ========================================================================
    # STEP 4: CLASSIFICATION METRICS - DEFAULT THRESHOLD 0.50
    # ========================================================================

    print_section_header("Classification Performance Metrics")

    # Calculate comprehensive metrics for both models at default 0.50 threshold
    calc = MetricsCalculator()
    hist_metrics = calc.calculate_metrics(y_test, y_proba_hist, y_pred_hist, 'HistGradientBoost')
    hf_metrics = calc.calculate_metrics(y_test, y_proba_hf, y_pred_hf, 'Neural Network')

    # Create comparison table of key metrics
    metrics_df = pd.DataFrame([hist_metrics, hf_metrics])
    metrics_display = metrics_df[[
        'Model', 'Accuracy', 'Precision', 'Recall', 'Specificity', 'F1_Score', 'ROC_AUC', 'MCC'
    ]].copy()

    # Format numbers for readability
    for col in ['Accuracy', 'Precision', 'Recall', 'Specificity', 'F1_Score', 'ROC_AUC', 'MCC']:
        metrics_display[col] = metrics_display[col].apply(lambda x: f"{x:.4f}")

    print_subsection("Default Threshold (0.50) Performance Comparison")
    print(metrics_display.to_string(index=False))

    # Print detailed classification reports
    print_subsection("Detailed Classification Report: HistGradientBoosting")
    print(classification_report(y_test, y_pred_hist, target_names=['No Churn', 'Churn']))

    print_subsection("Detailed Classification Report: Neural Network")
    print(classification_report(y_test, y_pred_hf, target_names=['No Churn', 'Churn']))

    # ========================================================================
    # STEP 5: STATISTICAL SIGNIFICANCE TESTING
    # ========================================================================

    print_section_header("Statistical Significance Testing")

    analyzer = StatisticalAnalyzer()

    # Bootstrap confidence intervals for ROC-AUC
    print_subsection("Bootstrap Confidence Intervals for ROC-AUC (1000 iterations)")
    hist_bootstrap = analyzer.bootstrap_auc(y_test, y_proba_hist)
    hf_bootstrap = analyzer.bootstrap_auc(y_test, y_proba_hf)

    print(f"HistGradientBoost AUC: {hist_bootstrap['AUC_Mean']:.4f} ± {hist_bootstrap['AUC_StdDev']:.4f}")
    print(f"95% Confidence interval: [{hist_bootstrap['CI_Lower']:.4f}, {hist_bootstrap['CI_Upper']:.4f}]")
    print()
    print(f"Neural Network AUC: {hf_bootstrap['AUC_Mean']:.4f} ± {hf_bootstrap['AUC_StdDev']:.4f}")
    print(f"95% Confidence interval: [{hf_bootstrap['CI_Lower']:.4f}, {hf_bootstrap['CI_Upper']:.4f}]")

    # Paired t-test comparing F1 scores across thresholds
    print_subsection("Paired t-Test: F1-Score Comparison Across 50 Thresholds")
    ttest_results = analyzer.paired_ttest_analysis(y_test, y_proba_hist, y_proba_hf)

    print(f"HistGradientBoost Mean F1: {ttest_results['HistGB_Mean_F1']:.4f} ± {ttest_results['HistGB_Std_F1']:.4f}")
    print(f"Neural Network Mean F1: {ttest_results['NN_Mean_F1']:.4f} ± {ttest_results['NN_Std_F1']:.4f}")
    print(f"t-statistic: {ttest_results['t_statistic']:.4f}")
    print(f"p-value: {ttest_results['p_value']:.6f}")

    if ttest_results['p_value'] < 0.05:
        print(f"Result: Statistically significant (p < 0.05)")
        print(f"Interpretation: Neural Network demonstrates superior performance")
    else:
        print(f"Result: Not statistically significant (p >= 0.05)")

    # Cross-validation on test set (additional validation)
    print_subsection("Cross-Validation Score Analysis (5-fold)")
    cv_hist = cross_val_score(hist_model, X_test, y_test, cv=5, scoring='roc_auc')
    cv_hf = cross_val_score(hf_model, X_test, y_test, cv=5, scoring='roc_auc')

    print(f"HistGradientBoost CV ROC-AUC: {cv_hist.mean():.4f} ± {cv_hist.std():.4f}")
    print(f"Fold scores: {[f'{x:.4f}' for x in cv_hist]}")
    print()
    print(f"Neural Network CV ROC-AUC: {cv_hf.mean():.4f} ± {cv_hf.std():.4f}")
    print(f"Fold scores: {[f'{x:.4f}' for x in cv_hf]}")

    # ========================================================================
    # STEP 6: PROFIT OPTIMIZATION ANALYSIS
    # ========================================================================

    print_section_header("Profit Maximization Analysis")

    # Define cost matrices based on business problem
    BASIC_COSTS = {
        'TP': 539.41,      # Retain a churner: CLV ($2,331.36) - retention cost ($160) - salvage value
        'FP': -160.00,     # Wasted retention attempt (cost of contact)
        'FN': -2331.36,    # Lost customer (their full lifetime value)
        'TN': 0            # Correct non-churn prediction (no action needed)
    }

    SEGMENT_COSTS = {
        'High': {
            'TP': 732.54,      # High-risk segment: higher value of retention
            'FP': -200.00,     # Higher investment in retention for high-risk
            'FN': -2331.36,
            'TN': 0
        },
        'Medium': {
            'TP': 539.41,      # Standard cost
            'FP': -160.00,
            'FN': -2331.36,
            'TN': 0
        },
        'Low': {
            'TP': 228.14,      # Low-risk segment: lower value of retention
            'FP': -5.00,       # Minimal investment in retention for low-risk
            'FN': -2331.36,
            'TN': 0
        }
    }

    print_subsection("Business Context and Cost Structure")
    print(f"Customer Lifetime Value (CLV): $2,331.36")
    print(f"Standard retention cost: $160.00")
    print(f"TP value (retain churner): $539.41 (CLV minus cost)")
    print(f"FP cost (wasted contact): -$160.00")
    print(f"FN cost (lost customer): -$2,331.36")
    print(f"Class weight ratio (FN:FP): 14.57:1")

    # Threshold optimization for HistGradientBoost (basic strategy)
    print_subsection("Threshold Optimization: HistGradientBoost (Basic Strategy)")
    hist_results_basic, hist_opt_thresh_basic = MetricsCalculator.generate_profit_function(
        y_test, y_proba_hist, BASIC_COSTS
    )
    hist_basic_profit = max([r['profit'] for r in hist_results_basic])

    print(f"Optimal threshold: {hist_opt_thresh_basic:.3f}")
    print(f"Maximum profit: ${hist_basic_profit:,.2f}")

    # Threshold optimization for Neural Network (basic strategy)
    print_subsection("Threshold Optimization: Neural Network (Basic Strategy)")
    hf_results_basic, hf_opt_thresh_basic = MetricsCalculator.generate_profit_function(
        y_test, y_proba_hf, BASIC_COSTS
    )
    hf_basic_profit = max([r['profit'] for r in hf_results_basic])

    print(f"Optimal threshold: {hf_opt_thresh_basic:.3f}")
    print(f"Maximum profit: ${hf_basic_profit:,.2f}")

    # ========================================================================
    # STEP 7: SEGMENT-BASED STRATEGY
    # ========================================================================

    print_subsection("Segment-Based Strategy Analysis")

    def segment_analysis(y_proba, segment_costs, model_name):
        """
        Perform profit optimization within customer risk segments.

        Segmentation Strategy:
        - High-Risk (P > 0.60): Heavy retention investment for high-churn customers
        - Medium-Risk (0.30 < P ≤ 0.60): Moderate retention for medium churners
        - Low-Risk (P ≤ 0.30): Light touch for unlikely churners

        Each segment uses its own optimal threshold and cost structure.
        """
        # Define segments based on predicted churn probability
        masks = {
            'High': y_proba > 0.6,                              # Top 20% risk
            'Medium': (y_proba > 0.3) & (y_proba <= 0.6),      # Middle 40% risk
            'Low': y_proba <= 0.3                               # Bottom 40% risk
        }

        results = []
        total_profit = 0

        for seg_name in ['High', 'Medium', 'Low']:
            seg_mask = masks[seg_name]
            y_seg, y_proba_seg = y_test[seg_mask], y_proba[seg_mask]

            # Skip if segment too small for reliable metric calculation
            if len(y_seg) < 10:
                continue

            # Optimize threshold within this segment using segment-specific costs
            seg_results, best_thresh = MetricsCalculator.generate_profit_function(
                y_seg, y_proba_seg, segment_costs[seg_name]
            )

            # Extract best result
            seg_profit = max([r['profit'] for r in seg_results])
            best_result = [r for r in seg_results if r['profit'] == seg_profit][0]

            # Store segment results
            results.append({
                'Model': model_name,
                'Segment': seg_name,
                'Sample_Count': seg_mask.sum(),
                'Threshold': best_thresh,
                'Profit': seg_profit,
                'Precision': best_result['precision'],
                'Recall': best_result['recall'],
                'F1_Score': best_result['f1']
            })

            total_profit += seg_profit

        return results, total_profit

    # Apply segmentation to both models
    hist_seg_results, hist_seg_profit = segment_analysis(y_proba_hist, SEGMENT_COSTS, 'HistGradientBoost')
    hf_seg_results, hf_seg_profit = segment_analysis(y_proba_hf, SEGMENT_COSTS, 'Neural Network')

    print(f"HistGradientBoost Segment-Based Total Profit: ${hist_seg_profit:,.2f}")
    print(f"Neural Network Segment-Based Total Profit: ${hf_seg_profit:,.2f}")

    # ========================================================================
    # STEP 8: RESULTS SUMMARY
    # ========================================================================

    print_section_header("Summary of Results")

    # Create strategy comparison table
    strategy_summary = pd.DataFrame({
        'Model': ['HistGradientBoost', 'HistGradientBoost', 'Neural Network', 'Neural Network'],
        'Strategy': ['Basic', 'Segment-Based', 'Basic', 'Segment-Based'],
        'Profit': [f"${hist_basic_profit:,.0f}", f"${hist_seg_profit:,.0f}",
                   f"${hf_basic_profit:,.0f}", f"${hf_seg_profit:,.0f}"]
    })

    print_subsection("Strategy Performance Comparison")
    print(strategy_summary.to_string(index=False))

    # Highlight best strategy
    print_subsection("Recommended Model and Strategy")
    print(f"Best performing model: Neural Network (Segment-Based)")
    print(f"Annual profit: ${hf_seg_profit:,.2f}")
    print(f"Improvement vs. baseline: {((hf_seg_profit/hist_basic_profit) - 1)*100:.1f}%")
    print(f"Tuning value gained: ${hf_seg_profit - hf_basic_profit:,.2f}")

    # ========================================================================
    # STEP 9: GENERATE OUTPUT REPORTS
    # ========================================================================

    print_section_header("Report Generation")

    # Save classification metrics to CSV
    metrics_report = pd.DataFrame([hist_metrics, hf_metrics])
    metrics_report.to_csv('classification_metrics_report.csv', index=False)
    print("Generated: classification_metrics_report.csv")

    # Save segment analysis results to CSV
    segment_report_data = []
    for results in [hist_seg_results, hf_seg_results]:
        for r in results:
            segment_report_data.append(r)

    segment_df = pd.DataFrame(segment_report_data)
    segment_df.to_csv('segment_analysis_metrics.csv', index=False)
    print("Generated: segment_analysis_metrics.csv")

    # Save strategy performance summary to CSV
    strategy_df = pd.DataFrame({
        'Model': strategy_summary['Model'],
        'Strategy': strategy_summary['Strategy'],
        'Profit': strategy_summary['Profit']
    })
    strategy_df.to_csv('strategy_performance.csv', index=False)
    print("Generated: strategy_performance.csv")

    print_section_header("Analysis Complete")
    print("\nAll analyses have been completed successfully.")
    print("Output files have been generated in the working directory.")


if __name__ == "__main__":
    main()



                                 CUSTOMER CHURN PREDICTION ANALYSIS                                 

                              Data Preparation and Feature Engineering                              

Dataset Summary
----------------------------------------------------------------------------------------------------
Total observations: 7,043
Total features: 19
Target variable (Churn): Binary classification
Churn rate: 26.54%
Class imbalance ratio: 2.77:1
Training set size: 5,634
Testing set size: 1,409

                                    Hyperparameter Optimization                                     

HistGradientBoosting Classifier Tuning
----------------------------------------------------------------------------------------------------
Grid search completed: 81 parameter combinations evaluated
Cross-validation strategy: 3-fold stratified
Optimal ROC-AUC score: 0.8424
Best parameters: {'l2_regularization': 1.0, 'learning_rate': 0.05, 'max_depth': 10, 'max_iter': 150}

Neural Ne